In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import multiprocessing
import swifter
import warnings
warnings.filterwarnings('ignore')

/Users/Georgios.Drakos/anaconda3/envs/bedswap-env/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
data = pd.DataFrame()
data['col1'] = np.random.normal(size = 15000000)
data['col2'] = np.random.normal(size = 15000000)
def myfunc(x,y): return y*(x**2+1)

In [3]:
%%time
df = data.apply(lambda row: myfunc(row.col1,row.col2), axis=1)

CPU times: user 5min 52s, sys: 1.9 s, total: 5min 53s
Wall time: 5min 57s


In [4]:
%%time 
ddata = dd.from_pandas(data, npartitions=2*multiprocessing.cpu_count())\
.map_partitions(lambda df: df.apply((lambda row: myfunc(row.col1,row.col2)), axis=1))\
.compute(scheduler='processes')

CPU times: user 1.25 s, sys: 1.67 s, total: 2.92 s
Wall time: 2min 55s


In [5]:
%%time 
ddata = dd.from_pandas(data, npartitions=2*multiprocessing.cpu_count())\
.map_partitions(lambda df: df.apply((lambda row: myfunc(row.col1,row.col2)), axis=1))\
.compute(scheduler='threads')

CPU times: user 5min 58s, sys: 4.8 s, total: 6min 3s
Wall time: 6min 6s


In [9]:
%%time 
ddata = dd.from_pandas(data, npartitions=2*multiprocessing.cpu_count())\
.map_partitions(lambda df: df.apply((lambda row: myfunc(row.col1,row.col2)), axis=1))\
.compute(scheduler='single-threaded')

CPU times: user 5min 40s, sys: 1.03 s, total: 5min 41s
Wall time: 5min 43s


In [10]:
%%time 
ddata = dd.from_pandas(data, npartitions=1)\
.map_partitions(lambda df: df.apply((lambda row: myfunc(row.col1,row.col2)), axis=1))\
.compute(scheduler='processes')

CPU times: user 1.43 s, sys: 3.03 s, total: 4.46 s
Wall time: 3min


In [11]:
%%time
df = data.swifter.apply(lambda row: myfunc(row.col1,row.col2), axis=1)

CPU times: user 137 ms, sys: 85.9 ms, total: 223 ms
Wall time: 111 ms


In [12]:
%%time
df = myfunc(data['col1'], data['col2']  )

CPU times: user 153 ms, sys: 72.3 ms, total: 225 ms
Wall time: 112 ms
